In [51]:
import jax
import jax.numpy as jnp
import numpy as np

from specq_jax.pulse import DragPulse, JaxBasedPulseSequence
from specq_jax.core import get_simulator, rotating_transmon_hamiltonian
from specq_dev.specq.qiskit import (
    get_circuit,
    get_ibm_service_and_backend,
    get_qubit_properties_from_backend,
)
import specq_dev.specq.shared as specq
from qiskit_experiments.library import ProcessTomography, InterleavedRB
from qiskit.quantum_info import average_gate_fidelity, random_unitary, Operator

In [5]:
instance = "utokyo-kawasaki/keio-internal/keio-students"
backend_name = "ibm_kawasaki"

service, backend = get_ibm_service_and_backend(instance, backend_name)

QubitInformationV3(unit='GHz', qubit_idx=0, anharmonicity=-0.20640003780600244, frequency=5.243549519417257, drive_strength=0.11218268029237063, date='2024-04-22 19:37:42')

In [49]:
PATH = "../tests/test_data/drag_0002/"
pulse_sequence = JaxBasedPulseSequence.from_file(PATH, pulses=[DragPulse])
exp_config = specq.ExperimentConfigV3.from_file(PATH)
qubit_info = get_qubit_properties_from_backend(backend, 0) # exp_config.qubits[0]
t_eval = jnp.linspace(0, exp_config.sequence_duration_dt * exp_config.device_cycle_time_ns, exp_config.sequence_duration_dt)
simulator = get_simulator(qubit_info, t_eval, rotating_transmon_hamiltonian)

key = jax.random.PRNGKey(0)
params = pulse_sequence.sample_params(key)
waveform = pulse_sequence.get_waveform(params)

unitaries = simulator(waveform)

/Users/porametpathumsoot/miniconda3/envs/specq-jax/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:66: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype)


In [8]:
qc = get_circuit(
    initial_state="0",
    waveforms=waveform,
    observable='Z',
    backend=backend,
    q_idx=0,
    add_pulse=True,
    change_basis=False,
    add_measure=False,
    enable_MCMD=False,
)

qc.draw()

┌────────────────┐
q: ┤ pulse_sequence ├
   └────────────────┘

In [9]:
qpt = ProcessTomography(qc)

In [12]:
result = qpt.run(backend=backend).block_for_results()

In [16]:
# Print results
for res in result.analysis_results():
    print(res)

AnalysisResult
- name: state
- value: Choi([[ 0.02675 +0.j      , -0.028   -0.00625j , -0.043625-0.005875j,
       -0.0035  -0.01225j ],
      [-0.028   +0.00625j ,  0.97325 +0.j      ,  0.95175 +0.0265j  ,
        0.043625+0.005875j],
      [-0.043625+0.005875j,  0.95175 -0.0265j  ,  0.984   +0.j      ,
        0.05325 -0.002j   ],
      [-0.0035  +0.01225j ,  0.043625-0.005875j,  0.05325 +0.002j   ,
        0.016   +0.j      ]],
     input_dims=(2,), output_dims=(2,))
- quality: unknown
- extra: <11 items>
- device_components: ['Q0']
- verified: False
AnalysisResult
- name: completely_positive
- value: True
- quality: unknown
- extra: <11 items>
- device_components: ['Q0']
- verified: False
AnalysisResult
- name: trace_preserving
- value: False
- quality: unknown
- extra: <11 items>
- device_components: ['Q0']
- verified: False


In [50]:
average_gate_fidelity(result.analysis_results(0).value, Operator(np.array(unitaries[-1])))

0.9772502442646612

In [53]:
iRB = InterleavedRB(qc, physical_qubits=[0], lengths=[1], backend=backend)

QiskitError: 'Interleaved element circuit-260 could not be converted to Clifford.'